%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.graph_objects as go

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
import src.data_handler as data_handler
from src.processing.raw_data_cleaning import prepare_graph_for_participant

/home/julien-rsbrg/venv/venv_semantic_to_liking/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = data_handler.load_data()

== Load Data: start ==
== Load Data: end ==


In [4]:
participant_graph, _ = prepare_graph_for_participant(data=data, 
                                                     participant_id=1, 
                                                     sim_used="original")
participant_graph

Test function convert_table_to_graph
Data(x=[60, 2], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], x_names=[2], edge_attr_names=[1], y_names=[1], train_mask=[60], val_mask=[60])
validate: True
is undirected: True
has_self_loop: tensor(False)
end Test function convert_table_to_graph


/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdata["_sim"] = subdata["senenceBERT_mpnet_similarity"]
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_features_word1.rename(columns=col_renaming,inplace=True)
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

Data(x=[60, 2], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], x_names=[2], edge_attr_names=[1], y_names=[1], train_mask=[60], val_mask=[60])

In [5]:
participant_graph.x_names,participant_graph.edge_attr_names,participant_graph.num_nodes

(['liking', 'experience'], ['_sim'], 60)

In [6]:
# start with random clusters (later take cs = x[:,1] > 0)
cs = np.random.randint(0,2,size=participant_graph.num_nodes)
cs

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1])

assumes only 2 clusters, if there are others, they are omitted
can use torch_geometric.datasets.InfectionDataset for validation

params:
cluster_id_start
cluster_id_end

% what if inf? no path between the two... 

ALGO 1

distance_to_other_cluster_nodes = np.zeros((len(x),len(x))) 
weighted_distance_to_other_cluster_nodes = np.zeros((len(x),len(x)))

for node_i in cluster_id_start:
    for node_j in cluster_id_end:
        shortest_path_ij = shortest_path(node_i,node_j,graph) # get the nodes ids [], get the weights []
        is_cluster_end = shortest_path_ij[0] == cluster_id_end # if there is another cluster on the way before, it is omitted
        distance_to_cluster_end = np.where(is_cluster_end)[0][0]
        weighted_distance_to_cluster_end = sum(shortest_path_ij[:distance_to_cluster_end]

        distance_to_other_cluster_nodes[node_i, node_j] = distance_to_cluster_end
        weighted_distance_to_other_cluster_nodes[node_i, node_j] = weighted_distance_to_cluster_end

distance_to_other_cluster = np.min(distance_to_other_cluster_nodes,axis=1)
weighted_distance_to_other_cluster = np.min(weighted_distance_to_other_cluster_nodes,axis=1)

mean_distance_to_other_cluster = np.mean(distance_to_other_cluster)
mean_weighted_distance_to_other_cluster = np.mean(weighted_distance_to_other_cluster)


ALGO 2

Take random walks of length, length of the graph (or bounded in param)... and compute the average 


In [7]:
from torch_geometric.data import Data
from typing import Iterable

def shortest_path_djikstra(node_start:int,edge_index:np.array,edge_weight:Iterable,num_nodes:int|None = None):
    assert node_start <= np.max(np.max(edge_index),num_nodes), f"There is no edge connected to node_start or the number of nodes ({num_nodes}) given is wrong..."
    
    _num_nodes = num_nodes
    if num_nodes is None:
        _num_nodes = np.max(edge_index)+1
        
    distance_from_start = np.inf * np.ones(_num_nodes)
    distance_from_start[node_start] = 0

    shortest_path_from_start = {node_id:{"edge_index":[],"edge_weight":[]} for node_id in range(_num_nodes)}

    segments_to_check = [(0,node_start)]
    while len(segments_to_check):
        selected_segment = segments_to_check.pop(-1)
        sending_edges_mask = edge_index[0,:] == selected_segment[1]

        receiving_neighbor_nodes = edge_index[1,sending_edges_mask] 
        neighbor_distances = edge_weight[sending_edges_mask]

        for edge_local_id,node_id in enumerate(receiving_neighbor_nodes):
            current_path_distance = selected_segment[0] + neighbor_distances[edge_local_id]
            if distance_from_start[node_id] > current_path_distance:
                distance_from_start[node_id] =  current_path_distance
                segments_to_check.append((distance_from_start[node_id],node_id))
                shortest_path_from_start[node_id]["edge_index"] = shortest_path_from_start[selected_segment[1]]["edge_index"] + [[selected_segment[1],node_id]] 
                shortest_path_from_start[node_id]["edge_weight"] = shortest_path_from_start[selected_segment[1]]["edge_weight"] + [neighbor_distances[edge_local_id]] 
    return distance_from_start, shortest_path_from_start

edge_index = np.array([[0,0,0,0,1,2,2,3,4],
                       [1,2,3,4,6,3,5,5,6]])
edge_weight = np.array([2,2,4,1,2,1,5,1,2])
true_values = np.array([0,2,2,3,1,4,3])

assert np.allclose(true_values,shortest_path_djikstra(0,edge_index,edge_weight)[0])


In [8]:
shortest_path_djikstra(0,edge_index,edge_weight)

(array([0., 2., 2., 3., 1., 4., 3.]),
 {0: {'edge_index': [], 'edge_weight': []},
  1: {'edge_index': [[0, 1]], 'edge_weight': [2]},
  2: {'edge_index': [[0, 2]], 'edge_weight': [2]},
  3: {'edge_index': [[0, 2], [2, 3]], 'edge_weight': [2, 1]},
  4: {'edge_index': [[0, 4]], 'edge_weight': [1]},
  5: {'edge_index': [[0, 2], [2, 3], [3, 5]], 'edge_weight': [2, 1, 1]},
  6: {'edge_index': [[0, 4], [4, 6]], 'edge_weight': [1, 2]}})

In [15]:
import copy 

def shortest_path_to_cluster_djikstra(node_start:int,edge_index:np.array,edge_weight:Iterable,cluster:Iterable[bool],num_nodes:int|None = None,exclude_itself:bool=True):
    """
    shortest path from node_start to a node belonging to cluster (set to True).

    Parameters
    ----------
    - exclude_itself: (bool)
        If True and node_start in cluster, removes node_start from the cluster identification and computes the distance to the nodes of the same cluster
    """
    _cluster = copy.copy(cluster)
    if exclude_itself:
        _cluster[node_start] = False

    node_ids_cluster = np.where(_cluster)[0]
    
    if node_start in node_ids_cluster:
        return 0, {node_start: {'edge_index': [], 'edge_weight': []}}
    else:
        distances_from_start, shortest_path_from_start = shortest_path_djikstra(node_start,edge_index,edge_weight,num_nodes=num_nodes)
        closest_node_id_cluster = node_ids_cluster[np.argmin(distances_from_start[node_ids_cluster])]

        shortest_path_to_cluster = {closest_node_id_cluster: shortest_path_from_start[closest_node_id_cluster]}
        return distances_from_start[closest_node_id_cluster],shortest_path_to_cluster

def mean_shortest_distance_to_cluster_djikstra(node_start:int,edge_index:np.array,edge_weight:Iterable,cluster:Iterable[bool],num_nodes:int|None = None,exclude_itself:bool=True):
    """
    mean shortest distance from node_start to a node belonging to cluster (set to True).
    """
    _cluster = copy.copy(cluster)
    if exclude_itself:
        _cluster[node_start] = False

    node_ids_cluster = np.where(_cluster)[0]
    distances_from_start, _ = shortest_path_djikstra(node_start,edge_index,edge_weight,num_nodes=num_nodes)
    return np.mean(distances_from_start[node_ids_cluster])

print(shortest_path_to_cluster_djikstra(0,edge_index,edge_weight,cluster = [True,True,False,True,False,True,True],exclude_itself=True))
print(mean_shortest_distance_to_cluster_djikstra(0,edge_index,edge_weight,cluster = [False,True,False,True,False,True,True]))

(2.0, {1: {'edge_index': [[0, 1]], 'edge_weight': [2]}})
3.0


In [ ]:
participant_ids = data["participant"].unique()
n_participants = len(participant_ids)
for i in participant_ids:
    print(f"participant / n_participant: {i} / {n_participants}")
    participant_graph, _ = prepare_graph_for_participant(data=data, 
                                                        participant_id=1, 
                                                        sim_used="original")
    x = pd.DataFrame(participant_graph.x,columns=participant_graph.x_names)
    cluster_has_been_experienced = (x["experience"] > 0).values
    edge_index = participant_graph.edge_index.numpy() # check to remove
    edge_weight = participant_graph.edge_attr.numpy() # check to remove

    node_data = {"min_to_exp":[],"min_to_not_exp":[],"mean_to_exp":[],"mean_to_not_exp":[],"experienced":[]}
    for node_id in range(participant_graph.num_nodes):
        print(f"node_id + 1 / num_nodes : {node_id} / {participant_graph.num_nodes} ")
        # Careful: suboptimal, compute twice the shortest distance
        min_to_exp = shortest_path_to_cluster_djikstra(node_id,edge_index,edge_weight,cluster_has_been_experienced)
        mean_to_exp = mean_shortest_distance_to_cluster_djikstra(node_id,edge_index,edge_weight,cluster_has_been_experienced)

        min_to_not_exp = shortest_path_to_cluster_djikstra(node_id,edge_index,edge_weight,~cluster_has_been_experienced)
        mean_to_not_exp = mean_shortest_distance_to_cluster_djikstra(node_id,edge_index,edge_weight,~cluster_has_been_experienced)

        node_data["min_to_exp"].append(min_to_exp)
        node_data["mean_to_exp"].append(mean_to_exp)
        node_data["min_to_not_exp"].append(min_to_not_exp)
        node_data["mean_to_not_exp"].append(mean_to_not_exp)
        node_data["experienced"].append(cluster_has_been_experienced[node_id])


participant / n_participant: 1 / 112
Test function convert_table_to_graph
Data(x=[60, 2], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], x_names=[2], edge_attr_names=[1], y_names=[1], train_mask=[60], val_mask=[60])
validate: True
is undirected: True
has_self_loop: tensor(False)
end Test function convert_table_to_graph


/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdata["_sim"] = subdata["senenceBERT_mpnet_similarity"]
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_features_word1.rename(columns=col_renaming,inplace=True)
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing/raw_data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

KeyboardInterrupt: 